# Spam Classification

<td>
    <a href="#" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>

In [3]:
import os
import tarfile
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Downloading data

In [8]:
DATA_URL = 'https://spamassassin.apache.org/old/publiccorpus/'
HAM_URL = DATA_URL + '20030228_easy_ham.tar.bz2'
SPAM_URL = DATA_URL + '20030228_spam.tar.bz2'
DATA_PATH = 'data'

def download_data(ham_url=HAM_URL, spam_url=SPAM_URL, data_path=DATA_PATH):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    for filename, url in (("ham.tar.bz2", ham_url), ("spam.tar.bz2", spam_url)):
        path = os.path.join(data_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=data_path)
        tar_bz2_file.close()

In [9]:
download_data()

## Looking for data